# VR score

> This computes the VR score from pluripotency and velocity metrics

In [ ]:
#| default_exp score

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| export
def compute_distdeg(input_df, dim_name = ["phateepi_1","phateepi_2"], split_by="curatedCLUST", 
    knn=30, distance_metric = "manhattan", scale = True, inv_degree=True,
    plot=False, rmv_outliers=True):
    "Function that computes distances and degrees of cells within a cluster."
    #Results will get scaled within the cluster level so it handles them as independent entities
    output_df = pd.DataFrame(columns=dim_name+["dist","deg"])
    for i in input_df[split_by].unique():
        print(i)
        df_dist = input_df.loc[input_df[split_by]==i,dim_name]
        #Knn param is key and must be tunned according to the size of the cluster
        G = graphtools.Graph(df_dist.values, knn=knn, decay=None, distance=distance_metric)
        # Return distances by using shortest_path
        np_dist = G.shortest_path(distance="data")
        #Compute median distance to all cells for al cells -> min should be a centroid
        df_dist["dist"] = np.median(np_dist, axis=1)
        df_dist["deg"] = G.to_pygsp().d
        if rmv_outliers:
            #Replace distant outliers with median value -> DONE to smoothen landscape at the very edges of a cluster
            df_dist["dist"] = np.where(
                                df_dist["dist"]>np.percentile(df_dist["dist"],99), 
                                df_dist["dist"].median(), 
                                df_dist["dist"])
        if scale:
            df_dist["dist_scaled"] = MinMaxScaler().fit_transform(X = df_dist[["dist"]])
            df_dist["inv_deg_scaled"] = MinMaxScaler().fit_transform(X = 1/df_dist[["deg"]])
        if plot:
            #Plot cluster cells on phate and colour by distance from centroid
            plt.scatter(df_dist[dim_name[0]],df_dist[dim_name[1]],c=df_dist["dist"])
            plt.show()
            plt.scatter(df_dist[dim_name[0]],df_dist[dim_name[1]],c=df_dist["deg"])
            plt.show()

        output_df = pd.concat([output_df, df_dist])

    return output_df

In [ ]:
#| export
def foo(): pass

In [ ]:
# test_eq(say_hi("Jeremy"), "Hi Jeremy!")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()